In [1]:
%matplotlib inline

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from data_generators import *
import glob
import cv2
import pickle
import h5py
import pandas as pd
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow.python.client import device_lib

from keras.models import Sequential, load_model
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation, Dropout, Flatten, Dense, Lambda, SpatialDropout2D
from keras.layers import Conv2D, ELU
from keras.optimizers import rmsprop, Adam
import keras.backend.tensorflow_backend as KTF
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model

# SpatialDropout2D <==> data_format='channels_last'

Using TensorFlow backend.


In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2879432174968659316, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11323454260
 locality {
   bus_id: 1
 }
 incarnation: 16125217443312319976
 physical_device_desc: "device: 0, name: Tesla K40c, pci bus id: 0000:03:00.0, compute capability: 3.5"]

In [26]:
BATCH_SIZE, ROWS, COLS, CHANNELS = 32, 480, 640, 4

#inspired in https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf
def cnn_model():
    in_shape = (ROWS, COLS, CHANNELS)
    
    model = Sequential()
    
    model.add(Conv2D(32, (7, 7), padding='valid', 
                     kernel_initializer='lecun_uniform',
                     strides=(2,2), input_shape=in_shape)
    )
    model.add(ELU())
    
    
    model.add(Conv2D(48, (5, 5), padding='valid', 
                     kernel_initializer='lecun_uniform',
                     strides=(2,2))
    )
    model.add(ELU())
    
    model.add(SpatialDropout2D(0.5))
    
    model.add(Conv2D(64, (5, 5), padding='valid', 
                     kernel_initializer='lecun_uniform',
                     strides=(2,2))
    )
    model.add(ELU())

    model.add(Conv2D(64, (3, 3), padding='valid', 
                     kernel_initializer='lecun_uniform',
                     strides=(2,2))
    )
    model.add(ELU())

#     model.add(SpatialDropout2D(0.5))
    
#     model.add(Conv2D(128, (3, 3), padding='valid', 
#                      kernel_initializer='lecun_uniform',
#                      strides=(2,2))
#     )
#     model.add(ELU())
    
    
    
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(ELU())
#     model.add(Dropout(0.25))

    model.add(Dense(128))
    model.add(ELU())
    model.add(Dropout(0.5))

    model.add(Dense(64))
    model.add(ELU())
#     model.add(Dropout(0.5))

    model.add(Dense(1))

#     optimizer = rmsprop(lr=0.0001, decay=1e-6)
    optimizer = Adam(
#         lr=0.001, 
#         beta_1=0.9, 
#         beta_2=0.999, 
#         epsilon=None, 
#         decay=0.0, 
        amsgrad=bool(1)
    )
    model.compile(optimizer=optimizer, loss='mse')
    
    return model

In [27]:
thresh = 0.05
suffix = "%03dp"%(np.round(100.0*thresh))
train_files = sorted(glob.glob('./train_%s/*.npz'%suffix))
total_n_train = len(train_files)

# test_files = sorted(glob.glob('./test/*.npz'))

n_train, n_valid, n_test = int(0.8*total_n_train), int(0.19*total_n_train), int(0.01*total_n_train)

print(n_train, n_valid, n_test)
train_steps = int(np.ceil(float(n_train) / float(BATCH_SIZE)))
valid_steps = int(np.ceil(float(n_valid) / float(BATCH_SIZE)))
test_steps = int(np.ceil(float(n_test) / float(BATCH_SIZE)))

train_ids = np.random.choice(total_n_train, size=n_train, replace=False)
avail_ids = np.setdiff1d(np.arange(total_n_train), train_ids)
test_ids = np.random.choice(avail_ids, size=n_test, replace=False)
valid_ids = np.setdiff1d(avail_ids, test_ids)

train_data = [train_files[i] for i in train_ids]
valid_data = [train_files[i] for i in valid_ids]
test_data = [train_files[i] for i in test_ids]

train_gen = train_generator(train_data, BATCH_SIZE)
valid_gen = train_generator(valid_data, BATCH_SIZE)
test_gen = train_generator(test_data, BATCH_SIZE)


16319 3875 203


In [28]:
output_dir = 'training_output'
os.makedirs(output_dir, exist_ok=True)

model = cnn_model()


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

model_fname = os.path.join(output_dir, 'best_val_model_%s.h5'%suffix)
mc = ModelCheckpoint(model_fname, monitor='val_loss', 
                     mode='min', save_best_only=True)

model_fname = os.path.join(output_dir, 'best_train_model_%s.h5'%suffix)
mc = ModelCheckpoint(model_fname, monitor='loss', 
                     mode='min', save_best_only=True)

# test_gen = test_generator(test_files, BATCH_SIZE)

In [ ]:
fit_log = model.fit_generator(
    train_gen, 
    steps_per_epoch=train_steps, 
    validation_data=valid_gen,
    validation_steps=valid_steps,
    epochs=100,
    verbose=1,
    shuffle=True,
)

history_fname = os.path.join(output_dir, 'train_history_dict.pickle')
with open(history_fname, 'wb') as file_pi:
        pickle.dump(fit_log.history, file_pi)
        

model_fname = os.path.join(output_dir, 'speed_test_model_%s.h5'%suffix)
model.save(model_fname)

#scores = model.evaluate()

Epoch 1/100
509/510 [============================>.] - ETA: 0s - loss: 32.2832

In [8]:
os.makedirs('training_output', exist_ok=True)
model_fname = os.path.join('training_output', 'speed_test_model.h5')
model = load_model(model_fname)

In [ ]:
scores = model.evaluate_generator(test_gen, steps=test_steps, verbose=1)

In [ ]:
plot_model(model)

In [ ]:
fig = plt.figure()
ax = plt.subplot(1, 1, 1)
plt.plot(fit_log.history['loss'])
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(1, 1, 1)
plt.plot(fit_log.history['loss'])
plt.plot(fit_log.history['val_loss'])
plt.show()

In [ ]:
print(dir(model))

In [ ]:
# print(model.summary())
print(model.weights)
# plt.figure()
# plt.plot(model.summary())